# ZTF data: alerts and light curves

---

Run this notebook in the browser in `Google Colab`: 

<a href="https://colab.research.google.com/github/dmitryduev/kowalski/blob/master/nb/ztf.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>

<b>Tip:</b> if you are running this notebook in `Google Colab` and run out of disk space with the default runtime, try changing it to a GPU-accelerated one in `Runtime` -> `Change runtime type` -> `Hardware accelerator` -> `GPU`.

---

## ZTF alerts

See [here](https://zwickytransientfacility.github.io/ztf-avro-alert/) for a detailed description of the ZTF alert packet contents.

A dedicated `Kafka` consumer running on the Caltech's `Kowalski` instance listens to the ZTF alert stream. All alerts are saved into the database. To optimize data storage/querying and to remove redundant data, the alert data are dealt with as follows.

- Individual alerts minus the `prv_candidates` block are stored in the `ZTF_alerts` collection. Machine learning models are executed on individual alerts and the results are save to the `classifications` block.

- The contents of the `prv_candidates` blocks of individual alerts are stored in the `ZTF_alerts_aux` collection per `objectId`. No filtering is applied, however all duplicate entries originating from different alerts are removed. [In rare cases, there may be both a detection and a non-detection for the same value of `candidate.jd` due to uncertanties in the astrometry.] The data in `prv_candidates` are stored as a set meaning that they are not ordered. Additionally, the information on cross-matched sources from external catalogs (within $5"$) is saved to the `cross_matches` block.

Let us look at examples.

In [1]:
from IPython.core.display import display, HTML, JSON
import json

from penquins import Kowalski

In [4]:
# with open('secrets_penquins.json', 'r') as f:
#     secrets = json.load(f)

# k = Kowalski(username=secrets['kowalski']['username'], password=secrets['kowalski']['password'])

k = Kowalski(host='localhost', port=8000, protocol='http', username='admin', password='admin')

connection_ok = k.check_connection()
print(f'Connection OK: {connection_ok}')

Connection OK: True
